In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.gold_serving.dimension_runs
USING DELTA
AS
SELECT
  time,
  in_num,
  in_name,
  pk
FROM workspace.gold_staging.stg_runs;

MERGE INTO workspace.gold_serving.dimension_runs AS target
USING (
  SELECT * FROM (
    SELECT
      time,
      in_num,
      in_name,
      pk,
      ROW_NUMBER() OVER (
        PARTITION BY time, in_num
        ORDER BY time DESC
      ) AS rn
    FROM workspace.gold_staging.stg_runs
  ) sub
  WHERE rn = 1
) AS source
ON target.in_num = source.in_num
AND target.
